![ros_control_logo](images/logos/logo_ros-controls.png) ![pal_logo](images/logos/palroboticslogo.png) ![denis_logo](images/logos/denis.png)

![](images/logos/image_ROSDevDay21.png)

# From ros_control to ros2_control #

A brief, hands-on introduction to ros_control.
We will use ROS Noetic to show you
* when and how to use ros_control,
* how to implement a Hardware Abstraction Layer (HAL),
* how to implement a controller for your ros_control setup.

Building on your newfound ros_control knowledge, we move to ROS2 Foxy and

* introduce you to the concepts of ros2_control,
* show you how to implement a System component (sub-HAL),
* demonstrate how to implement a controller for a ros2_control setup.

Buckle up, there's a lot to learn today!

We are going to use a 2 degrees of freedom robot arm for demoing. 
![rrbot](images/rrbot.png)

But here's a *small* list of robots that use ros_control
![ROS1 ros_control robots](images/ros_control_montage.jpg)

## WHO ARE WE

### Bence Magyar, PhD in Robotics, Lead Software Engineer at FiveAI <div style="width: 100px;">![bence_image](images/bence.jpeg)</div> 
maintainer at ros_control and ros2_control

### ‪Denis Štogl, PhD in Robotics, Robotics Consultant at Stogl Robotics <div style="width: 100px;">![denis_image](images/denis.jpeg)</div> 
maintainer at ros2_control


## ros_control in a nutshell

![](images/overview.png)

* RobotHW is also referred to as the Hardware Abstraction Layer (HAL).
* Controllers claim resources from controller manager (safe)
* Controllers write set commands on claimed joint interface handles


![interfaces_and_controllers](images/roscontrol_interfaces_controllers.png)

Today we'll explain you how to write a RobotHW and a controller and what steps are important to pay attention to.

For the first set of demos we are going to use the ROS Noetic environment. Make sure to source first.

## Implementing a ros_control RobotHW 

[Include paragraphs with code (C++, Python, XML) or command lines if necessary, like the following three]

## Implementing a controller for ros_control

[Describe here step by step how to do and replicate your experiment/program/code result. The steps must be clear and reproducible by the audience. The clearer they are, the easier it will be for you to follow along the notes during your presentation]

[Include paragraphs with code (C++, Python, XML) or command lines if necessary, like the following three]

[ex. of command line]

In [ ]:
$ cd /home/user/simulation_ws/src

[ex. of launch file]

In [ ]:
<?xml version="1.0"?>
<launch>
    <!-- maps inside the folder summit_xl_navigation/maps -->
    <arg name="map_file" default="map1/map1.yaml"/>
    <arg name="prefix" default="summit_xl_"/>
    <arg name="frame_id" default="$(arg prefix)map"/>

    <!-- Run the map server -->
    <node name="map_server" pkg="map_server" type="map_server" args="$(find summit_xl_localization)/maps/$(arg map_file)">
        <param name="frame_id" value="$(arg frame_id)"/>
    </node>

</launch>

[ex. of C++ code]

In [ ]:
if ( !inliers->indices.empty() )
{
    //Extract inliers
    pcl::ExtractIndices<PointT> extract;
    extract.setInputCloud(inputCloud);
    extract.setIndices(inliers);
    extract.setNegative(false);
    extract.filter(*cylinderCloud);
    return true;
}

[Include images of the expected results to help audience understand your point, like the following image]

![](images/rosject_images/sim7.png)

[If you need extra code already available to the audience (for example, in case you want to use an arm robot with MoveIt and you need the MoveIt configuration already done) you can upload the code and ROS packages to the catkin_ws/src directory using the IDE->Upload option]

[If you need to provide a dataset, follow instructions below about how to create/upload a dataset and provide it to audience]

# ROS2 and ros\_control = ros2\_control

In ros2_control, the functionalities of a HAL and RobotHW are separated. The role of HAL is taken by the ResourceManager which can manage multiple Sensor, System or Actuator components which serve as self-contained hardware drivers, essentially equivalent to ros_control's RobotHW.

![](images/components_architecture.png)


For the following demos we'll use the ROS2 Foxy environment, make sure to run

Checking rrbot

```
ros2 launch rrbot_description view_robot.launch.py
```

![rrbot rviz](images/rrbot_gui.png)

## Implementing a system component in ros2_control

## Implementing a controller in ros2_control

Creating a new controller 

Denis: abbreviate this?

```C++
class RRBotController : public controller_interface::ControllerInterface
{
public:
  RRBotController();

  controller_interface::return_type init(const std::string & controller_name) override;

  controller_interface::InterfaceConfiguration command_interface_configuration() const override;

  controller_interface::InterfaceConfiguration state_interface_configuration() const override;

  CallbackReturn on_configure(const rclcpp_lifecycle::State & previous_state) override;

  CallbackReturn on_activate(const rclcpp_lifecycle::State & previous_state) override;

  CallbackReturn on_deactivate(const rclcpp_lifecycle::State & previous_state) override;

  controller_interface::return_type update() override;

protected:
  std::vector<std::string> joint_names_;
  std::string interface_name_;

  // TODO(anyone): replace the state and command message types
  // Command subscribers and Controller State publisher
  using ControllerCommandMsg = control_msgs::msg::JointJog;

  rclcpp::Subscription<ControllerCommandMsg>::SharedPtr command_subscriber_ = nullptr;
  realtime_tools::RealtimeBuffer<std::shared_ptr<ControllerCommandMsg>> input_command_;

  using ControllerStateMsg = control_msgs::msg::JointControllerState;
  using ControllerStatePublisher = realtime_tools::RealtimePublisher<ControllerStateMsg>;

  rclcpp::Publisher<ControllerStateMsg>::SharedPtr s_publisher_;
  std::unique_ptr<ControllerStatePublisher> state_publisher_;
};
```

Testing the new controller: add controller and configuration file to launch file

`ros2 launch rrbot_bringup rrbot_with_rrbot_controller.launch.py`

Check which controllers are loaded now and which interfaces are claimed

```
ros2 control list_controllers
```

```
ros2 control list_hardware_interfaces
```

Examine the state of the controller on the state topic or the joint states topic

```ros2 topic echo /rrbot_controller/state```


```ros2 topic echo /joint_states```


## [Concluding remarks]


[Write here your conclusions]

# References

* [ros_control paper in the Journal of Open Source Software](https://joss.theoj.org/papers/10.21105/joss.00456)
* ros_control
** http://wiki.ros.org/ros_control
** http://wiki.ros.org/ros_controllers (select individual controllers)
** https://github.com/ros-controls/roadmap/blob/master/documentation_resources.md
* ros2_control
** https://ros-controls.github.io/control.ros.org/
** https://github.com/ros-controls/ros2_control
** https://github.com/ros-controls/ros2_controllers
** https://github.com/ros-controls/ros2_control_demos

# Extra ROSject Documentation

[You can delete all the information in this section if you don't need it]